In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot  as plt
import seaborn as sn
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pickle
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import mean_absolute_error, mean_squared_error
import csv

In [ ]:
df = pd.read_csv("/home/growlt199/Downloads/manufacturing_project/data/flight_processed_data.csv",index_col=0)

In [ ]:
df.head()

In [ ]:
#done label encoding in flight column
label_encoder = LabelEncoder()
df['flight'] = label_encoder.fit_transform(df['flight'])

In [ ]:
df.head()

In [ ]:
categorical_columns = ['airline', 'departure_time','arrival_time','source_city','destination_city']

# Apply label encoding to each categorical column
for column in categorical_columns:
    df[column] = label_encoder.fit_transform(df[column])

In [ ]:
df.head()

In [25]:
#divide into x and y
X = df.drop('price',axis=1)
y = df['price']

In [26]:
X.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left
0,4,1408,2,2,0,5,5,0,2.17,1
1,4,1387,2,1,0,4,5,0,2.33,1
2,0,1213,2,1,0,1,5,0,2.17,1
3,5,1559,2,4,0,0,5,0,2.25,1
4,5,1549,2,4,0,4,5,0,2.33,1


In [27]:
y.head()

0    5953
1    5953
2    5956
3    5955
4    5955
Name: price, dtype: int64

In [28]:
#use sklearn train test spilt for spilt our data in parts like training data and testing data 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state =101, shuffle=True)

In [29]:
X_train.shape

(239063, 10)

In [30]:
X_test.shape

(59766, 10)

In [31]:
#scaling in x train and y train for model building and testing
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [32]:
print(X_train_scaled[:10])

[[ 1.0353104   0.86278963  0.81262437  1.47248555  0.1932792  -1.76404208
  -0.33642355 -0.67083373  0.74897036  0.14617432]
 [-1.14827428 -0.45210827  0.81262437 -0.8072832   0.1932792  -0.61640925
   1.3831308  -0.67083373 -0.35186278 -0.14920206]
 [-1.14827428 -0.72868216  1.38403905 -0.8072832  -2.32201227 -1.19022567
  -0.90960833  1.49068235 -1.43020124  1.32767982]
 [-1.14827428 -0.5294552  -1.47303435  1.47248555 -2.32201227  1.10503999
  -0.33642355 -0.67083373 -1.30085686 -1.69992803]
 [ 1.0353104   0.87450886 -1.47303435 -0.23734101  0.1932792  -1.19022567
   1.3831308  -0.67083373  0.12755498 -1.10917528]
 [-0.60237811 -0.05599821  0.24120969  0.90254336  0.1932792  -0.61640925
  -0.33642355 -0.67083373 -0.6569468   0.07233023]
 [ 1.0353104   1.0479535   1.38403905 -0.8072832   0.1932792  -0.61640925
   0.23676123  1.49068235 -0.37576337 -1.25686346]
 [ 0.48941423  0.51824419  0.81262437 -0.23734101  0.1932792   0.53122357
  -0.33642355 -0.67083373  0.7025751   0.81077116]


In [33]:
print(X_test_scaled[:10])

[[ 1.0353104   1.05498504 -0.33020499 -0.23734101  0.1932792  -1.19022567
  -1.48279311  1.49068235  0.40873841  1.47536801]
 [-1.14827428 -0.56695675  0.81262437  1.47248555  0.1932792   0.53122357
   0.23676123 -0.67083373  0.45653959 -0.14920206]
 [-1.14827428 -0.75680832  0.24120969  0.90254336  0.1932792  -0.04259284
  -1.48279311  1.49068235  0.09240705  0.58923888]
 [-0.60237811 -0.17084669 -0.33020499  0.90254336  0.1932792  -0.61640925
  -1.48279311 -0.67083373 -0.91422964 -1.33070756]
 [-0.05648194 -1.7435677  -0.90161967 -0.8072832  -2.32201227  0.53122357
   1.3831308  -0.67083373 -1.43020124  0.81077116]
 [ 1.0353104   0.89560348 -0.33020499  0.90254336 -2.32201227  0.53122357
  -1.48279311 -0.67083373 -1.32475745  1.17999163]
 [-0.05648194 -1.98967158 -1.47303435 -0.8072832   0.1932792  -0.61640925
  -0.33642355 -0.67083373 -0.45730656 -1.25686346]
 [ 1.0353104   0.92607348 -0.90161967 -0.8072832   0.1932792   0.53122357
   0.80994601  1.49068235  2.0381964  -0.29689024]


# building basic neural network

In [36]:
model = Sequential()

# Add input layer and the first hidden layer
model.add(Dense(64, activation='relu', input_dim=10))

# Add additional hidden layers
model.add(Dense(32, activation='relu'))
model.add(Dense(8, activation='relu'))

# Add output layer
model.add(Dense(1))

In [37]:
import time
start_time = time.time()

model.compile(optimizer='adam', loss='mean_squared_error',metrics=['mae'])

#print model summary
print(model.summary())

# fiting data in neural network
model.fit(X_train, y_train, epochs=10, batch_size=32)

end_time = time.time()
execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 64)                704       
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dense_6 (Dense)             (None, 8)                 264       
                                                                 
 dense_7 (Dense)             (None, 1)                 9         
                                                                 
Total params: 3,057
Trainable params: 3,057
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
7471/7471 [==============================] - 5s 610us/step - loss: 334220224.0000 - mae: 14277.4150
Epoch 2/10
7471/7471 [==============================] - 5s 626us/ste

In [38]:
pwd

'/home/growlt199/Downloads/flight_price_prediction/model_training'

In [39]:
cd Neural_network

/home/growlt199/Downloads/flight_price_prediction/model_training/Neural_network


In [40]:
y_pred = model.predict(X_test)

1868/1868 [==============================] - 1s 450us/step


In [41]:
# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)

print("MAE:", mae)
print("MSE:", mse)

MAE: 3476.394143196879
MSE: 31678702.83808789


In [68]:
#with open('nn_128_to_1_T7_layers', 'wb') as file:
  #  pickle.dump(model, file)

In [42]:
# with open('nn_128_to_1_T7_layers', 'rb') as file:
#     model = pickle.load(file)

# # Make predictions on the new data
# predictions = model.predict(X_test)

# use other activation fuction 

In [ ]:
activation_functions = [
    'linear',
    'relu',
    'sigmoid',
    'tanh',
    'softplus'
]

with open('activation_nn_results_100_epochs.csv', 'w', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(['activation1','activation2','mae','mse','execution_time']) 
            
            
for act in activation_functions:
    for act1 in activation_functions:
        model = Sequential()

        # Add input layer and the first hidden layer
        model.add(Dense(32, activation=act, input_dim=10))

        # Add additional hidden layers
        model.add(Dense(8, activation=act1))
        # model.add(Dense(8, activation='relu'))
        # Add output layer
        model.add(Dense(1))
        
        print(f'{act} and {act1} as activation functions ')
        
        import time
        start_time = time.time()

        model.compile(optimizer='adam', loss='mean_squared_error',metrics=['mae'])

        #print model summary
        print(model.summary())

        # fiting data in neural network
        model.fit(X_train, y_train, epochs=100, batch_size=128)
        
        #predictions
        y_pred = model.predict(X_test)

        # Calculate Mean Absolute Error (MAE)
        mae = mean_absolute_error(y_test, y_pred)

        # Calculate Mean Squared Error (MSE)
        mse = mean_squared_error(y_test, y_pred)
        
        end_time = time.time()
        execution_time = end_time - start_time
        print("Execution time:", execution_time, "seconds")
        
        print("MAE:", mae)
        print("MSE:", mse)
        
        with open('activation_nn_results_100_epochs.csv', 'a', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow([act,act1,mae,mse,execution_time]) 
            
        #file name 
        file_name = 'nn_32_8_1_' + act + '_' + act1 +'_100_epochs'
        
        #dump in pickle files
        with open(file_name, 'wb') as file:
            pickle.dump(model, file)
            

linear and linear as activation functions 
Model: "sequential_71"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_215 (Dense)           (None, 32)                352       
                                                                 
 dense_216 (Dense)           (None, 8)                 264       
                                                                 
 dense_217 (Dense)           (None, 1)                 9         
                                                                 
Total params: 625
Trainable params: 625
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
1868/1868 [==============================] - 1s 600us/step - loss: 488881024.0000 - mae: 17996.9512
Epoch 2/100
1868/1868 [==============================] - 1s 610us/step - loss: 450948928.0000 - mae: 17954.7734
Epoch 3/100
1868/1868 [========================

1868/1868 [==============================] - 1s 539us/step - loss: 46144784.0000 - mae: 4535.6299
Epoch 68/100
1868/1868 [==============================] - 1s 536us/step - loss: 46188628.0000 - mae: 4537.8433
Epoch 69/100
1868/1868 [==============================] - 1s 547us/step - loss: 46271596.0000 - mae: 4552.6372
Epoch 70/100
1868/1868 [==============================] - 1s 553us/step - loss: 46131656.0000 - mae: 4532.4106
Epoch 71/100
1868/1868 [==============================] - 1s 555us/step - loss: 46109240.0000 - mae: 4534.8936
Epoch 72/100
1868/1868 [==============================] - 1s 626us/step - loss: 46114064.0000 - mae: 4532.4717
Epoch 73/100
1868/1868 [==============================] - 1s 522us/step - loss: 46104092.0000 - mae: 4531.0850
Epoch 74/100
1868/1868 [==============================] - 1s 560us/step - loss: 46209768.0000 - mae: 4539.0747
Epoch 75/100
1868/1868 [==============================] - 1s 551us/step - loss: 46043876.0000 - mae: 4526.3242
Epoch 76/100
1

                                                                 
 dense_222 (Dense)           (None, 8)                 264       
                                                                 
 dense_223 (Dense)           (None, 1)                 9         
                                                                 
Total params: 625
Trainable params: 625
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
1868/1868 [==============================] - 1s 603us/step - loss: 936097344.0000 - mae: 20742.4922
Epoch 2/100
1868/1868 [==============================] - 1s 601us/step - loss: 935622784.0000 - mae: 20731.0703
Epoch 3/100
1868/1868 [==============================] - 1s 685us/step - loss: 935160768.0000 - mae: 20719.8926
Epoch 4/100
1868/1868 [==============================] - 1s 604us/step - loss: 934698752.0000 - mae: 20708.7285
Epoch 5/100
1868/1868 [==============================] - 1s 623us/step - loss: 93423686

1868/1868 [==============================] - 1s 579us/step - loss: 915415232.0000 - mae: 20237.7598
Epoch 32/100
1868/1868 [==============================] - 1s 582us/step - loss: 914738048.0000 - mae: 20221.0527
Epoch 33/100
1868/1868 [==============================] - 1s 628us/step - loss: 914062272.0000 - mae: 20204.3281
Epoch 34/100
1868/1868 [==============================] - 1s 589us/step - loss: 913386496.0000 - mae: 20187.6133
Epoch 35/100
1868/1868 [==============================] - 1s 610us/step - loss: 912712000.0000 - mae: 20170.8887
Epoch 36/100
1868/1868 [==============================] - 1s 630us/step - loss: 912037504.0000 - mae: 20154.1523
Epoch 37/100
1868/1868 [==============================] - 1s 597us/step - loss: 911363008.0000 - mae: 20137.4531
Epoch 38/100
1868/1868 [==============================] - 1s 615us/step - loss: 910691648.0000 - mae: 20120.6992
Epoch 39/100
1868/1868 [==============================] - 1s 570us/step - loss: 910017728.0000 - mae: 20104.0

 dense_227 (Dense)           (None, 32)                352       
                                                                 
 dense_228 (Dense)           (None, 8)                 264       
                                                                 
 dense_229 (Dense)           (None, 1)                 9         
                                                                 
Total params: 625
Trainable params: 625
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
1868/1868 [==============================] - 1s 570us/step - loss: 495731328.0000 - mae: 17954.5469
Epoch 2/100
1868/1868 [==============================] - 1s 599us/step - loss: 452956384.0000 - mae: 17985.9668
Epoch 3/100
1868/1868 [==============================] - 1s 549us/step - loss: 440216832.0000 - mae: 17757.3730
Epoch 4/100
1868/1868 [==============================] - 1s 626us/step - loss: 420103904.0000 - mae: 17290.7930
Epoch 5/100
1868/1868

...

1868/1868 [==============================] - 1s 595us/step - loss: 29917852.0000 - mae: 3511.6892
Epoch 36/100
1868/1868 [==============================] - 1s 630us/step - loss: 29798410.0000 - mae: 3501.1331
Epoch 37/100
1868/1868 [==============================] - 1s 584us/step - loss: 29779894.0000 - mae: 3497.8081
Epoch 38/100
1868/1868 [==============================] - 1s 574us/step - loss: 29562014.0000 - mae: 3480.9663
Epoch 39/100
1868/1868 [==============================] - 1s 574us/step - loss: 29528474.0000 - mae: 3479.1814
Epoch 40/100
1868/1868 [==============================] - 1s 574us/step - loss: 29503846.0000 - mae: 3480.8120
Epoch 41/100
1868/1868 [==============================] - 1s 585us/step - loss: 29446144.0000 - mae: 3473.8552
Epoch 42/100
1868/1868 [==============================] - 1s 579us/step - loss: 29397056.0000 - mae: 3472.6858
Epoch 43/100
1868/1868 [==============================] - 1s 550us/step - loss: 29301324.0000 - mae: 3464.5176
Epoch 44/100
1

Trainable params: 625
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
1868/1868 [==============================] - 1s 554us/step - loss: 905175616.0000 - mae: 19971.5898
Epoch 2/100
1868/1868 [==============================] - 1s 575us/step - loss: 774206144.0000 - mae: 17000.6055
Epoch 3/100
1868/1868 [==============================] - 1s 657us/step - loss: 625004608.0000 - mae: 16122.5225
Epoch 4/100
1868/1868 [==============================] - 1s 545us/step - loss: 507232512.0000 - mae: 16508.7129
Epoch 5/100
1868/1868 [==============================] - 1s 583us/step - loss: 372164992.0000 - mae: 14834.1045
Epoch 6/100
1868/1868 [==============================] - 1s 609us/step - loss: 227946928.0000 - mae: 11804.7930
Epoch 7/100
1868/1868 [==============================] - 1s 561us/step - loss: 118426624.0000 - mae: 8282.3877
Epoch 8/100
1868/1868 [==============================] - 1s 560us/step - loss: 64691512.0000 - mae: 

1868/1868 [==============================] - 1s 565us/step - loss: 29066778.0000 - mae: 3359.4692
Epoch 38/100
1868/1868 [==============================] - 1s 610us/step - loss: 29208100.0000 - mae: 3362.7373
Epoch 39/100
1868/1868 [==============================] - 1s 544us/step - loss: 29222492.0000 - mae: 3362.5208
Epoch 40/100
1868/1868 [==============================] - 1s 572us/step - loss: 29031514.0000 - mae: 3352.6755
Epoch 41/100
1868/1868 [==============================] - 1s 549us/step - loss: 28879508.0000 - mae: 3345.9541
Epoch 42/100
1868/1868 [==============================] - 1s 556us/step - loss: 29019848.0000 - mae: 3349.7434
Epoch 43/100
1868/1868 [==============================] - 1s 648us/step - loss: 28937422.0000 - mae: 3344.4556
Epoch 44/100
1868/1868 [==============================] - 1s 621us/step - loss: 28929090.0000 - mae: 3346.2195
Epoch 45/100
1868/1868 [==============================] - 1s 577us/step - loss: 28665274.0000 - mae: 3332.3345
Epoch 46/100
1

1868/1868 [==============================] - 1s 565us/step - loss: 893682944.0000 - mae: 19693.5410
Epoch 74/100
1868/1868 [==============================] - 1s 574us/step - loss: 893024448.0000 - mae: 19676.8125
Epoch 75/100
1868/1868 [==============================] - 1s 540us/step - loss: 892367040.0000 - mae: 19660.1211
Epoch 76/100
1868/1868 [==============================] - 1s 534us/step - loss: 891709760.0000 - mae: 19643.3770
Epoch 77/100
1868/1868 [==============================] - 1s 620us/step - loss: 891052864.0000 - mae: 19626.7109
Epoch 78/100
1868/1868 [==============================] - 1s 557us/step - loss: 890397632.0000 - mae: 19610.0078
Epoch 79/100
1868/1868 [==============================] - 1s 558us/step - loss: 889741568.0000 - mae: 19593.3535
Epoch 80/100
1868/1868 [==============================] - 1s 555us/step - loss: 889087232.0000 - mae: 19576.6738
Epoch 81/100
1868/1868 [==============================] - 1s 524us/step - loss: 888433792.0000 - mae: 19559.9

Total params: 625
Trainable params: 625
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
1868/1868 [==============================] - 1s 576us/step - loss: 922002624.0000 - mae: 20398.7852
Epoch 2/100
1868/1868 [==============================] - 1s 582us/step - loss: 852694272.0000 - mae: 18652.2324
Epoch 3/100
1868/1868 [==============================] - 1s 591us/step - loss: 752827904.0000 - mae: 16562.3789
Epoch 4/100
1868/1868 [==============================] - 1s 584us/step - loss: 650256704.0000 - mae: 15934.8525
Epoch 5/100
1868/1868 [==============================] - 1s 620us/step - loss: 557898752.0000 - mae: 16043.4941
Epoch 6/100
1868/1868 [==============================] - 1s 576us/step - loss: 451854368.0000 - mae: 14816.0801
Epoch 7/100
1868/1868 [==============================] - 1s 560us/step - loss: 328573536.0000 - mae: 12676.0850
Epoch 8/100
1868/1868 [==============================] - 1s 566us/step - loss: 22